In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
folder_path='C:/Users/RSHIRINI/OneDrive - United Nations/Desktop/DSS/DATA COLLECTOR/datacollector_received_quest/recieved_quests'

#check all xlsx files in the folder

xlsx_files = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]
xlsx_files

['algeria-pop - Population_10_a.xlsx',
 'algeria-pop - Population_10_b.xlsx',
 'algeria-pop - Population_11.xlsx',
 'algeria-pop - Population_12_a.xlsx',
 'algeria-pop - Population_12_b.xlsx',
 'algeria-pop - Population_13_a.xlsx',
 'algeria-pop - Population_13_b.xlsx',
 'algeria-pop - Population_14.xlsx',
 'algeria-pop - Population_15.xlsx',
 'algeria-pop - Population_16.xlsx',
 'algeria-pop - Population_17.xlsx',
 'algeria-pop - Population_18.xlsx',
 'algeria-pop - Population_18_a.xlsx',
 'algeria-pop - Population_18_b.xlsx',
 'algeria-pop - Population_19_a.xlsx',
 'algeria-pop - Population_19_b.xlsx',
 'algeria-pop - Population_1_a.xlsx',
 'algeria-pop - Population_1_b.xlsx',
 'algeria-pop - Population_2.xlsx',
 'algeria-pop - Population_20_a.xlsx',
 'algeria-pop - Population_20_b.xlsx',
 'algeria-pop - Population_21_a.xlsx',
 'algeria-pop - Population_21_b.xlsx',
 'algeria-pop - Population_22.xlsx',
 'algeria-pop - Population_23_a.xlsx',
 'algeria-pop - Population_23_b.xlsx',
 'alg

In [3]:
#read in all files and combine into a single dataframe
dataframes = []
log_file_path = os.path.join(folder_path, "processing_log.txt")

# Open the log file in write mode
with open(log_file_path, "w", encoding="utf-8") as log_file:
    for file in xlsx_files:
        df_raw = pd.read_excel(os.path.join(folder_path, file), header=None, dtype=str)

        #Find the row nums where col zero has words "index"
        header_rows = df_raw.index[df_raw[0] == 'index'].tolist()

        # Get the value of these rows to be used as headers
        header_1_values = df_raw.iloc[header_rows[0]].dropna()
        header_2_values = df_raw.iloc[header_rows[1]].dropna()


        # for dataframe containing data: slice dataframe 1 (index=1) and set header
        df_data = df_raw[df_raw[0] == '1'].copy()
        # Set the columns using the 1st header we found
        df_data.columns = header_1_values
        #reshape the data dataframe from wide to long
        #id_vars are all colums that are not integers (years)
        id_variables = [col for col in header_1_values if not col.isdigit()]

        df_data=df_data.melt(id_vars=id_variables, var_name='السنة', value_name='العدد')
        df_data.drop(columns=['index'], inplace=True)


        # for the source dataframe
        df_source = df_raw[df_raw[0] == '2'].copy()
        # drop all the empty columns on the right side immediately.
        df_source = df_source[header_2_values.index]
        # Set the columns using the 2nd header we found
        df_source.columns = header_2_values
        df_source.drop(columns=['index'], inplace=True)

        #merge the source with the data left on المؤشر, السنة
        df_merged=pd.merge(df_data, df_source, on=['السنة','المؤشر', 'الدولة'], how='left')

        # #append to data list
        dataframes.append(df_merged)

        # Prepare log entry
        log_entry = (
            f"{'='*30}\n"
            f"SUCCESS: {file} finished processing.\n"
            f"Preview of merged data (head 2):\n"
            f"{df_merged.head(2).to_string()}\n"
            f"{'='*30}\n\n"
        )

        # Print to console
        print(log_entry)
        
        # Write to text file
        log_file.write(log_entry)
        
print(f"Logging complete. Summary saved to: {log_file_path}")

# Combine all
final_df = pd.concat(dataframes, ignore_index=True)
final_df.to_excel('final_dataset.xlsx', index=False)

SUCCESS: algeria-pop - Population_10_a.xlsx finished processing.
Preview of merged data (head 2):
                       المؤشر   الدولة السنة  العدد                المصدر
0  حالات الطلاق المسجلة (عدد)  الجزائر  2010  49845  مصلحة الحالة المدنية
1  حالات الطلاق المسجلة (عدد)  الجزائر  2011  54826  مصلحة الحالة المدنية


SUCCESS: algeria-pop - Population_10_b.xlsx finished processing.
Preview of merged data (head 2):
                         المؤشر   الدولة السنة العدد المصدر
0  معدل الطلاق (لكل 1،000 نسمة)  الجزائر  2010   NaN    NaN
1  معدل الطلاق (لكل 1،000 نسمة)  الجزائر  2011   NaN    NaN


SUCCESS: algeria-pop - Population_11.xlsx finished processing.
Preview of merged data (head 2):
                                                                                    المؤشر   الدولة الفئة العمرية السنة العدد                المصدر
0  معدلات الخصوبة العمرية التفصيلية  (عدد المواليد لكل 1,000 امرأة في الفئة العمرية 15-49)  الجزائر         15-19  2010   9.1  مصلحة الحالة المدنية
1  معد

In [4]:
dataframes[0]

,المؤشر,الدولة,السنة,العدد,المصدر
0,حالات الطلاق المسجلة (عدد),الجزائر,2010,49845,مصلحة الحالة المدنية
1,حالات الطلاق المسجلة (عدد),الجزائر,2011,54826,مصلحة الحالة المدنية
2,حالات الطلاق المسجلة (عدد),الجزائر,2012,54985,مصلحة الحالة المدنية
3,حالات الطلاق المسجلة (عدد),الجزائر,2013,57461,مصلحة الحالة المدنية
4,حالات الطلاق المسجلة (عدد),الجزائر,2014,60844,مصلحة الحالة المدنية
5,حالات الطلاق المسجلة (عدد),الجزائر,2015,59909,مصلحة الحالة المدنية
6,حالات الطلاق المسجلة (عدد),الجزائر,2016,62128,مصلحة الحالة المدنية
7,حالات الطلاق المسجلة (عدد),الجزائر,2017,65637,مصلحة الحالة المدنية
8,حالات الطلاق المسجلة (عدد),الجزائر,2018,65690,مصلحة الحالة المدنية
9,حالات الطلاق المسجلة (عدد),الجزائر,2019,65967,مصلحة الحالة المدنية


In [ ]:
# #read in all files and combine into a single dataframe
# dataframes = []
# for file in xlsx_files:
#     df = pd.read_excel(os.path.join(folder_path, file))
#     #find the first row that is all empty (NaN)
#     separator_iloc = df[df.isnull().all(axis=1)].index[0]

#     #for data dataframe
#     df_data=df.iloc[1:separator_iloc , :].copy().reset_index(drop=True)
#     #reshape the data dataframe from wide to long
#     #id_vars are all colums that are not integers (years)
#     id_variables = [col for col in df_data.columns if not col.isdigit()]

#     df_data=df_data.melt(id_vars=id_variables, var_name='السنة', value_name='العدد')
#     #convert the 'السنة' column to numeric
#     df_data['السنة'] = df_data['السنة'].astype(int)
    

#     #for the source dataframe
#     # Find the header row for the 'source' table (df2).
#     source_header=df.iloc[separator_iloc + 1, :3].tolist()
#     #read in data
#     df_source=df.iloc[separator_iloc + 2:separator_iloc + 18, :3].copy().reset_index(drop=True)
#     #set the header
#     df_source.columns=source_header
#     df_source['السنة'] = df_source['السنة'].astype(int)
    
#     #merge the source with the data left on المؤشر, السنة
#     df_merged=pd.merge(df_data, df_source, on=['السنة'], how='left')

#     # #append to data list
#     dataframes.append(df_merged)
    

### do the same by using empty row to divide the two dataframes

In [ ]:
# #read in all files and combine into a single dataframe
# dataframes = []
# for file in xlsx_files:
#     df = pd.read_excel(os.path.join(folder_path, file))
#     #find the first row that is all empty (NaN)
#     separator_iloc = df[df.isnull().all(axis=1)].index[0]

#     #for data dataframe
#     df_data=df.iloc[1:separator_iloc , :].copy().reset_index(drop=True)
#     #reshape the data dataframe from wide to long
#     #id_vars are all colums that are not integers (years)
#     id_variables = [col for col in df_data.columns if not col.isdigit()]

#     df_data=df_data.melt(id_vars=id_variables, var_name='السنة', value_name='العدد')
#     #convert the 'السنة' column to numeric
#     df_data['السنة'] = df_data['السنة'].astype(int)
    

#     #for the source dataframe
#     # Find the header row for the 'source' table (df2).
#     source_header=df.iloc[separator_iloc + 1, :3].tolist()
#     #read in data
#     df_source=df.iloc[separator_iloc + 2:separator_iloc + 18, :3].copy().reset_index(drop=True)
#     #set the header
#     df_source.columns=source_header
#     df_source['السنة'] = df_source['السنة'].astype(int)
    
#     #merge the source with the data left on المؤشر, السنة
#     df_merged=pd.merge(df_data, df_source, on=['السنة'], how='left')

#     # #append to data list
#     dataframes.append(df_merged)
    